In [1]:
import flickrapi
import webbrowser

# API keys
API_KEY = "db4c4d646adf494007498f2ffde82ea6"
API_SECRET = "26cad008ca7b167f"

# Code for authenticating the Flickr API
flickr = flickrapi.FlickrAPI(API_KEY, API_SECRET)

if not flickr.token_valid(perms="read"):
    flickr.get_request_token(oauth_callback="oob")
    authorize_url = flickr.auth_url(perms="read")
    webbrowser.open_new_tab(authorize_url)
    verifier = str(input("Verifier code: "))
    flickr.get_access_token(verifier)

In [2]:
import datetime
import pandas as pd
# Get list of photos
curr_time_utc = datetime.datetime.now(datetime.timezone.utc)
five_years_ago_timestamp = (curr_time_utc - datetime.timedelta(days=3*365)).timestamp()
photo_data_filename = "photo_data.csv"
west_coast_bbox = "-124.763068,32.534231,-114.134458,49.002494"
east_coast_bbox = "-87.634938,24.523096,-66.949895,47.459686"
attributes = ["date_upload", "date_taken", "last_update", "owner_name", "geo", "views"]
extracted_attributes = ["dateupload", "datetaken", "lastupdate", "ownername",
                        "latitude", "longitude", "views"]
from os.path import exists
if not exists(photo_data_filename):
    photos_dict = {}
    for attribute in extracted_attributes:
        photos_dict[attribute] = []
    num_pages = 10
    safe_search_level = 1
    has_geo = 1
    for i in range(1, num_pages + 1):
        photos_metadata = flickr.photos.search(format="parsed-json", min_upload_date=five_years_ago_timestamp,
                                               min_taken_date=five_years_ago_timestamp, bbox=west_coast_bbox,
                                               safe_search=safe_search_level, has_geo=has_geo,
                                               extras=",".join(attributes))
        photos_metadata2 = flickr.photos.search(format="parsed-json", min_upload_date=five_years_ago_timestamp,
                                               min_taken_date=five_years_ago_timestamp, bbox=east_coast_bbox,
                                               safe_search=safe_search_level, has_geo=has_geo,
                                               extras=",".join(attributes))
        for i in range(len(photos_metadata["photos"]["photo"])):
            for attribute in extracted_attributes:
                photos_dict[attribute].append(photos_metadata["photos"]["photo"][i][attribute])
                photos_dict[attribute].append(photos_metadata2["photos"]["photo"][i][attribute])
    photo_data = pd.DataFrame.from_dict(photos_dict)
    photo_data = photo_data.sample(frac=1).reset_index(drop=True)
    print(photo_data)
    photo_data.to_csv(photo_data_filename, index=False)
else:
    photo_data = pd.read_csv(photo_data_filename)

In [3]:
import folium
import math
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
loc_data_sample_filename = "location_data.csv"
place_data = photo_data[["latitude", "longitude"]].apply(pd.to_numeric)
place_data_values = place_data.values
loc_map = folium.Map(location=place_data.values[0], zoom_start=6)
if not exists(loc_data_sample_filename):
    rev_geocoder = Nominatim(user_agent="reverseGeocoder")
    reverse = RateLimiter(rev_geocoder.reverse, min_delay_seconds=2)
    loc_dict = {"latitude": [], "longitude": [], "city": [], "county": [], "state": []}
    locations = [reverse(p) for p in place_data_values]
    for loc in locations:
        if loc.raw["address"]["country"] != "United States":
            continue
        if "town" in loc.raw["address"]:
            loc_dict["city"].append(loc.raw["address"]["town"])
        elif "city" in loc.raw["address"]:
            loc_dict["city"].append(loc.raw["address"]["city"])
        elif "village" in loc.raw["address"]:
            loc_dict["city"].append(loc.raw["address"]["village"])
        else:
            loc_dict["city"].append("")
        if "county" in loc.raw["address"]:
            loc_dict["county"].append(loc.raw["address"]["county"])
        else:
            loc_dict["county"].append("")
        if "state" in loc.raw["address"]:
            loc_dict["state"].append(loc.raw["address"]["state"])
        else:
            loc_dict["state"].append("")
        loc_dict["latitude"].append(loc.raw["lat"])
        loc_dict["longitude"].append(loc.raw["lon"])
    loc_data = pd.DataFrame.from_dict(loc_dict)
    loc_data.to_csv(loc_data_sample_filename, index=False)
    
# loc_data = pd.read_csv(loc_data_sample_filename)
# popup_msg_format_str = "{}, {}"
# for i, row in loc_data.iterrows():
#     if type(row["city"]) == float and math.isnan(row["city"]):
#         popup_msg = popup_msg_format_str.format(row["county"], row["state"])
#     else:
#         popup_msg = popup_msg_format_str.format(row["city"], row["state"])
#     folium.map.Marker(location=[loc_data["latitude"][i], loc_data["longitude"][i]],
#                       popup=popup_msg).add_to(loc_map)
# loc_map